## Currents Analysis
This notebook processes and visualizes Salish Sea NEMO currents. Results are obtained from the Nowcast system through the ERDDAP server using the `xarray` library.

In [1]:
import numpy as np
import xarray as xr
import datetime as dtm
import dateutil.parser as dparser
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import copy

from salishsea_tools import viz_tools

#%matplotlib inline
#mpl.rcParams.update({'font.size': 12})
#mpl.rcParams["axes.formatter.useoffset"] = False

### Functions
These will be moved to a module.

In [56]:
def tidal_filter(record, winlen=39, method='box', index_name='time'):
    '''
    '''
    
    record.ndim
    
    # Preallocate filtered record
    filtered = copy.deepcopy(record)
    
    # Window length
    w = (winlen - 1) // 2
    
    # Construct weight vector
    weight = np.zeros(w, dtype=int)
    
    # Select filter method
    if method is 'doodson':
        # Doodson bandpass filter (winlen must be 39)
        weight[[1, 2, 5, 6, 10, 11, 13, 16, 18]] = 1
        weight[[0, 3, 8]] = 2
        centerval = 0
    elif method is 'box':
        # Box filter
        weight[:] = 1
        centerval = 1
    else:
        raise ValueError('Invalid filter method: {}'.format(method))
    
    # Loop through record
    for i, val in enumerate(record[index_name]):
        
        # Adjust window length for end cases
        W = min(i, w, record.shape[0]-i-1)
        Weight = weight[:W]
        Weight = np.append(Weight[::-1], np.append(centerval, Weight))
        Weight = (Weight/sum(Weight, centerval))
        
        for dim in range(record.ndim - 1):
            Weight = Weight[:, np.newaxis]
        
        # Apply mean over window length
        filtered[i, ...] = np.sum(record[i-W:i+W+1, ...] * Weight, axis=0)
    
    return filtered

In [3]:
def plot_wind(ax, x_in, y_in, u_in, v_in, coords='grid'):
    ''' Plot wind forcing on lon-lat projection over NEMO domain
    '''
    
    # Assign coordinate vectors
    if coords is 'grid':
        x, y = np.meshgrid(x_in, y_in)
        u, v = viz_tools.rotate_vel(u_in, v_in, origin='map')
    elif coords is 'map':
        x = x_in - 360
        y = y_in
        u = u_in
        v = v_in
    else:
        raise ValueError('Invalid coords value: {}'.format(coords))
    
    # Plot wind vectors
    Q = ax.quiver(x[::2, ::2], y[::2, ::2], u[::2, ::2], v[::2, ::2],
                  color='r', edgecolor='r', scale=20, linewidth=1, headwidth=6)
    
    return Q

In [4]:
def plot_horz_currents(ax, x_in, y_in, u_in, v_in, coords='grid'):
    ''' Plot horizontal currents on lon-lat projection over NEMO domain
    '''
    
    # Unstagger velocities
    u, v = viz_tools.unstagger(np.ma.masked_values(u_in, 0), np.ma.masked_values(v_in, 0))
    
    # Assign coordinate vectors
    if coords is 'grid':
        x, y = np.meshgrid(x_in, y_in)
    elif coords is 'map':
        x = x_in
        y = y_in
        u, v = viz_tools.rotate_vel(u, v, origin='grid')
    else:
        raise ValueError('Invalid coords value: {}'.format(coords))
    
    # Plot current vectors
    Q = ax.quiver(x[1::5, 1::5], y[1::5, 1::5], u[::5,::5], v[::5,::5], scale=10)
    
    return Q

### Nowcast results processing

In [2]:
# Load Nowcast and GEM results from ERDDAP using xarray
GEM_OP    = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSaSurfaceAtmosphereFieldsV1')
GEM_grid  = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSaAtmosphereGridV1')
u_vel     = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSn3DuVelocity1hV1')
v_vel     = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSn3DvVelocity1hV1')
NEMO_grid = xr.open_dataset('https://salishsea.eos.ubc.ca/erddap/griddap/ubcSSnBathymetry2V1')
Bathy_path = '/ocean/bmoorema/research/MEOPAR/NEMO-forcing/grid/bathy_meter_SalishSea2.nc'

In [57]:
# Slice results objects
u_grid = u_vel.uVelocity.sel(time=slice('2016-02-29 00:30', '2016-03-03 00:30'), depth=slice(0, 1))
v_grid = v_vel.vVelocity.sel(time=slice('2016-02-29 00:30', '2016-03-03 00:30'), depth=slice(0, 1))

In [60]:
# Apply Doodson filter to currents and wind
u_filtered = tidal_filter(u_grid, method='doodson')
v_filtered = tidal_filter(v_grid, method='doodson')

u_wind_filtered = tidal_filter(GEM_OP.u_wind.sel(time=slice('2016-02-29 00:00', '2016-03-03 00:00')), method='doodson')
v_wind_filtered = tidal_filter(GEM_OP.v_wind.sel(time=slice('2016-02-29 00:00', '2016-03-03 00:00')), method='doodson')

### Visualization

In [70]:
# Generate figure panels
fig, ax = plt.subplots(1, 2, figsize=(15, 10))
init_time = dparser.parse('2016-03-01 00:30')
map_bounds = [-124, -122.7, 48.3, 49.7]

#Q2 = ax[0].quiver(GEM_grid.longitude[::3, ::3]-360, GEM_grid.latitude[::3, ::3],
#                  GEM_OP.u_wind.sel(time=init_time, method='nearest')[::3, ::3],
#                  GEM_OP.v_wind.sel(time=init_time, method='nearest')[::3, ::3],
#                  color='r', edgecolor='r', scale=20, linewidth=1, headwidth=6)
#Q2.set_UVC(GEM_OP.u_wind.sel(time=time, method='nearest')[::3, ::3],
               #GEM_OP.v_wind.sel(time=time, method='nearest')[::3, ::3])

# Unfiltered
viz_tools.plot_land_mask(ax[0], Bathy_path, coords='map', color='burlywood')
viz_tools.plot_coastline(ax[0], Bathy_path, coords='map')
u = u_grid.sel(time=init_time, depth=0, method='nearest')
v = v_grid.sel(time=init_time, depth=0, method='nearest')
u, v = viz_tools.unstagger(np.ma.masked_values(u, 0), np.ma.masked_values(v, 0))
u, v = viz_tools.rotate_vel(u, v, origin='grid')
Q1  = ax[0].quiver(NEMO_grid.longitude[1::5, 1::5], NEMO_grid.latitude[1::5, 1::5], u[::5,::5], v[::5,::5], scale=10)
Q1W = ax[0].quiver(GEM_grid.longitude[::5, ::5]-360, GEM_grid.latitude[::5, ::5],
                  GEM_OP.u_wind.sel(time=init_time, method='nearest')[::5, ::5],
                  GEM_OP.v_wind.sel(time=init_time, method='nearest')[::5, ::5],
                  color='r', edgecolor='r', scale=40, linewidth=1, headwidth=6)
time_text1 = ax[0].text(-123.95, 49.65, init_time.strftime('%a %Y-%m-%d %H:%M:%S'))
viz_tools.set_aspect(ax[0])
ax[0].set_xlim(map_bounds[0:2])
ax[0].set_ylim(map_bounds[2:4])
ax[0].grid()
ax[0].add_patch(mpl.patches.Rectangle((-123, 49.5), 0.3, 0.2, facecolor='white'))
plt.quiverkey(Q1, 0.87, 0.93, 1, '1 m/s')
plt.quiverkey(Q1W, 0.85, 0.87, 5, '5 m/s')

# Filtered
viz_tools.plot_land_mask(ax[1], Bathy_path, coords='map', color='burlywood')
viz_tools.plot_coastline(ax[1], Bathy_path, coords='map')
u = u_filtered.sel(time=init_time, depth=0, method='nearest')
v = v_filtered.sel(time=init_time, depth=0, method='nearest')
u, v = viz_tools.unstagger(np.ma.masked_values(u, 0), np.ma.masked_values(v, 0))
u, v = viz_tools.rotate_vel(u, v, origin='grid')
Q2  = ax[1].quiver(NEMO_grid.longitude[1::5, 1::5], NEMO_grid.latitude[1::5, 1::5], u[::5,::5], v[::5,::5], scale=10)
Q2W = ax[1].quiver(GEM_grid.longitude[::5, ::5]-360, GEM_grid.latitude[::5, ::5],
                  u_wind_filtered.sel(time=init_time, method='nearest')[::5, ::5],
                  v_wind_filtered.sel(time=init_time, method='nearest')[::5, ::5],
                  color='r', edgecolor='r', scale=40, linewidth=1, headwidth=6)
time_text2 = ax[1].text(-123.95, 49.65, init_time.strftime('%a %Y-%m-%d %H:%M:%S'))
viz_tools.set_aspect(ax[1])
ax[1].set_xlim(map_bounds[0:2])
ax[1].set_ylim(map_bounds[2:4])
ax[1].grid()
ax[1].add_patch(mpl.patches.Rectangle((-123, 49.5), 0.3, 0.2, facecolor='white'))
plt.quiverkey(Q2, 0.87, 0.93, 1, '1 m/s')
plt.quiverkey(Q2W, 0.85, 0.87, 5, '5 m/s')

def update_currents(t):
    time = (init_time + dtm.timedelta(hours=t)).strftime('%a %Y-%m-%d %H:%M:%S')
    u = u_grid.sel(time=time, depth=0, method='nearest')
    v = v_grid.sel(time=time, depth=0, method='nearest')
    u, v = viz_tools.unstagger(np.ma.masked_values(u, 0), np.ma.masked_values(v, 0))
    u, v = viz_tools.rotate_vel(u, v, origin='grid')
    Q1.set_UVC(u[::5,::5], v[::5,::5])
    Q1W.set_UVC(GEM_OP.u_wind.sel(time=time, method='nearest')[::5, ::5],
                GEM_OP.v_wind.sel(time=time, method='nearest')[::5, ::5])
    time_text1.set_text(time)
    u = u_filtered.sel(time=time, depth=0, method='nearest')
    v = v_filtered.sel(time=time, depth=0, method='nearest')
    u, v = viz_tools.unstagger(np.ma.masked_values(u, 0), np.ma.masked_values(v, 0))
    u, v = viz_tools.rotate_vel(u, v, origin='grid')
    Q2.set_UVC(u[::5,::5], v[::5,::5])
    Q2W.set_UVC(u_wind_filtered.sel(time=time, method='nearest')[::5, ::5],
                v_wind_filtered.sel(time=time, method='nearest')[::5, ::5])
    time_text2.set_text(time)
    return Q1, Q2, Q1W, Q2W, time_text1, time_text2

mywriter = animation.FFMpegWriter(fps=12, bitrate=600)
ani = animation.FuncAnimation(fig, update_currents, frames=24, blit=False)
ani.save('/home/bmoorema/Desktop/SalishSeaCurrents_2.mp4', writer=mywriter)
#1 month = 744 frames